  # Neural Network Training for the MtM approach

In [21]:
import numpy as np
from scipy.optimize import fsolve
import matplotlib.pyplot as plt 
from tqdm import tqdm
%load_ext autoreload
%autoreload 2
%aimport IS_misc_functions
from IS_misc_functions import *
from misc_functions import *
from actuarial_training import *
from MtM_training import *

import pandas as pd

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras import regularizers
from sklearn.preprocessing import MinMaxScaler, StandardScaler, RobustScaler, MaxAbsScaler,normalize
from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle
import json
import csv

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


Set the range of parameters for training the neural network

In [2]:
obligor_bounds = 10,100

First set parameters

In [3]:
N_Iter = 10000 # Number of Iterations
save_interval = 1000 # Save every... iterations in a file
B = 1 # Batch Size


# Number of States considered
S =17
n_obligors = 100

# Bounds for the parameters
ELGD_bounds = 0,0.6
EAD_bounds = 0,1
rho_bounds = 0.15,0.7
c_bounds = 0,0.1
g_bounds = 2,S
D_bounds = 1,10
obligor_bounds = 10,100

n_MC = 500000 # Number of Monte Carlo Samples

Apply the training procedure

In [4]:
transition_matrix = pd.read_csv("transition_matrix_SP.csv",delimiter = ",")
PDs = np.maximum(np.array(transition_matrix["D"])/100,1e-8)
SP_dict = trans_matrix_to_dict(transition_matrix)

In [5]:
with open("probs.txt") as tweetfile:
    probs = json.load(tweetfile)
probs = {float(k):float(v) for k,v in probs.items()}
PDs =  np.maximum(np.array([float(k) for k,v in probs.items()])/100,1e-10)
PD_distribution = np.array([float(v) for k,v in probs.items()])

In [6]:
g_vals = np.arange(g_bounds[0],g_bounds[1])
g_probs = np.flip(np.concatenate([[0.0001,0.0001],PD_distribution]))
g_probs = g_probs/np.sum(g_probs)


In [31]:
# import time
# D_bounds = 1,1
# obligor_bounds = 99,100
# t = time.time()
# LGD_Vector,EAD_Vector,rho_Vector,c_vector,g_Vector,D_vector = create_sample_MtM(ELGD_bounds,EAD_bounds,rho_bounds,c_bounds,g_bounds,D_bounds,obligor_bounds,B,
#                                    g_vals,g_probs)

# LGD_Vector = LGD_Vector[0,:]
# EAD_Vector = EAD_Vector[0,:]
# rho_Vector = rho_Vector[0,:]
# c_vector = c_vector[0,:]
# g_Vector = g_Vector[0,:].astype(int)
# D_vector = D_vector[0,:]


# GA_GM(PD = 0, ELGD =LGD_Vector[LGD_Vector>0],
#                                    A = EAD_Vector[EAD_Vector>0], M= D_vector[EAD_Vector>0],
#                       q=0.999,        
#                      r = r_nelson,
#                      T=1,
#                      g= np.array(g_Vector[g_Vector>0]),
#                      trans_prob=SP_dict["trans_prob"],#N times S matrix
#                      psi = 0.4, # Market Sharpe Ratio,
#                      rho = rho_Vector[EAD_Vector>0],
#                      c = c_vector[EAD_Vector>0],
#                     nu =0.25,               
#                      S = S, # Number of States of Rating
#                      default_only = False,
#                       LGD_constant = False)
# s = time.time()
# print(s-t)

In [7]:
for iteration_save in range(13,15):
    List_x = []
    List_y = []
    for iteration in tqdm(range(save_interval)):
        try:
            ELGD,EAD,rho,c,g,D = create_sample_MtM(ELGD_bounds,EAD_bounds,rho_bounds,c_bounds,g_bounds,D_bounds,obligor_bounds,B,
                                                   g_vals,g_probs)
            if (iteration % 2) == 0:
                ELGD[ELGD>0] = 0.1
            if (iteration % 2) == 1:
                ELGD[ELGD>0] = 0.45            
            Input = np.concatenate([ELGD,EAD,rho,c,g,D],axis =1)




            GA_999 = MC_IS_MtM(ELGD[0,:],EAD[0,:],rho[0,:],c[0,:],g[0,:],
                               SP_dict,D[0,:],r_nelson,
                               q=0.999,n = n_MC)
            List_x.append(Input[0])
            List_y.append(GA_999)
        except:
            pass # doing nothing on exception         

    pd.DataFrame(List_x).to_csv("csv/X_mtm_"+str(iteration_save)+".csv")
    pd.DataFrame(List_y).to_csv("csv/y_mtm_"+str(iteration_save)+".csv")

100%|██████████| 1000/1000 [43:16:21<00:00, 155.78s/it]  
